<a href="https://colab.research.google.com/github/xsmu/ba765-assignments/blob/main/Team_3_US_Mutual_funds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Analysis of US Mutual Funds Data**
Ziyue Zhou, Xiangshan Mu, Shrey Sood, Yifang He, Jiazhi Jia, Weiyang Bao

Data Source: https://www.kaggle.com/stefanoleone992/mutual-funds-and-etfs?select=Mutual+Funds.csv

### ***Introduction:***

As a well known, in the US, financial institutions own about 80% of the market value of the US broad-market. Investment advisers are the largest institutional owner of equities through mutual funds and other investment institutions. Therefore, for individial investors, choosing a right fund that fit your own investment style can help you to acheve asset growth.

In this study, Our goal is to come up with a general analysis of US mutual fund data, for example, which aspects have substantial impact on the performance of the fund. The dataset includes the general aspects of the fund, portfolio indicators, returns, and financial ratios. Also, we want to know what kind of strategies and categories are most popular in recent years. Which investment type (growth, value, blend, others) has produced the high returns in the last few years. The impact of rating on the mutual funds' profitability.


### ***Raw Data Profile:***

The data file contains 24,821 Mutual Funds with total 291 columns, including general aspects (as Total Net Assets, management company and size), portfolio indicators (as cash, stocks, bonds, and sectors), returns (as yeartodate, 2020-11) and financial ratios (as price/earning, Treynor and Sharpe ratios, alpha, and beta etc).






In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Below is the raw data without cleaning:
%%bigquery --project=ba775-project-team3
SELECT * FROM `ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
LIMIT 5

,fund_symbol,fund_extended_name,fund_family,inception_date,category,rating,return_rating,risk_rating,investment_strategy,investment_type,size_type,currency,fund_net_annual_expense_ratio,category_net_annual_expense_ratio,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertable,price_earnings_ratio,price_book_ratio,price_sales_ratio,price_cashflow_ratio,median_market_cap,sector_basic_materials,sector_consumer_cyclical,sector_financial_services,sector_real_estate,sector_consumer_defensive,sector_healthcare,sector_utilities,sector_communication_services,sector_energy,sector_industrials,sector_technology,bond_maturity,bond_duration,credit_us_government,credit_aaa,...,fund_alpha_5years,category_alpha_5years,fund_alpha_10years,category_alpha_10years,fund_beta_3years,category_beta_3years,fund_beta_5years,category_beta_5years,fund_beta_10years,category_beta_10years,fund_mean_annual_return_3years,category_mean_annual_return_3years,fund_mean_annual_return_5years,category_mean_annual_return_5years,fund_mean_annual_return_10years,category_mean_annual_return_10years,fund_r_squared_3years,category_r_squared_3years,fund_r_squared_5years,category_r_squared_5years,fund_r_squared_10years,category_r_squared_10years,fund_standard_deviation_3years,category_standard_deviation_3years,fund_standard_deviation_5years,category_standard_deviation_5years,fund_standard_deviation_10years,category_standard_deviation_10years,fund_sharpe_ratio_3years,category_sharpe_ratio_3years,fund_sharpe_ratio_5years,category_sharpe_ratio_5years,fund_sharpe_ratio_10years,category_sharpe_ratio_10years,fund_treynor_ratio_3years,category_treynor_ratio_3years,fund_treynor_ratio_5years,category_treynor_ratio_5years,fund_treynor_ratio_10years,category_treynor_ratio_10years
0,AHSAX,Alger Health Sciences Fund Class A,Alger,2002-04-30,Health,4,4,4,The investment seeks long-term capital appreci...,Growth,Large,USD,1.12,1.21,8.10,89.83,0.0,1.85,0.22,0.0,33.55,8.15,4.64,25.00,29063.48,0.00,0.0,0.46,0.0,0.0,98.94,0.0,0.0,0.0,0.6,0.00,None,None,None,None,...,7.71,0.02,9.06,0.08,0.85,0.01,0.95,0.01,0.80,0.01,1.29,0.01,1.35,0.01,1.33,0.01,54.37,0.61,46.11,0.52,41.78,0.5,19.57,0.2,19.88,0.19,16.88,0.17,0.71,0.01,0.75,0.0,0.91,0.01,15.16,0.11,14.68,0.08,18.82,0.17
1,AHSCX,Alger Health Sciences Fund Class C,Alger,2002-04-30,Health,4,4,4,The investment seeks long-term capital appreci...,Growth,Large,USD,1.89,1.21,8.10,89.83,0.0,1.85,0.22,0.0,33.55,8.15,4.64,25.00,29063.48,0.00,0.0,0.46,0.0,0.0,98.94,0.0,0.0,0.0,0.6,0.00,None,None,None,None,...,6.93,0.02,8.29,0.08,0.85,0.01,0.95,0.01,0.79,0.01,1.23,0.01,1.28,0.01,1.27,0.01,54.41,0.61,46.14,0.52,41.71,0.5,19.57,0.2,19.87,0.19,16.86,0.17,0.67,0.01,0.72,0.0,0.86,0.01,14.13,0.11,13.75,0.08,17.75,0.17
2,BHSRX,BlackRock Health Sciences Opportunities Portfo...,BlackRock,2011-09-11,Health,4,4,2,The investment seeks to provide long-term grow...,Blend,Large,USD,1.46,1.21,2.36,96.42,0.0,1.22,0.00,0.0,25.46,4.70,2.13,17.56,59092.98,0.59,0.0,0.53,0.0,0.0,98.89,0.0,0.0,0.0,0.0,0.00,None,None,None,None,...,4.61,0.02,9.31,0.08,0.76,0.01,0.79,0.01,0.73,0.01,1.13,0.01,0.99,0.01,1.31,0.01,65.69,0.61,58.04,0.52,53.56,0.5,15.86,0.2,14.71,0.19,13.68,0.17,0.76,0.01,0.72,0.0,1.10,0.01,15.21,0.11,12.83,0.08,20.88,0.17
3,DLHAX,Delaware Healthcare Fund Class A,Delaware Funds by Macquarie,2007-09-27,Health,3,3,3,The investment seeks maximum long-term capital...,Blend,Large,USD,1.27,1.21,0.00,100.00,0.0,0.00,0.00,0.0,17.88,3.07,2.17,10.85,31584.01,0.12,0.0,0.38,0.0,1.0,89.00,0.0,3.1,0.0,0.0,6.41,None,None,None,None,...,3.02,0.02,7.93,0.08,0.87,0.01,0.92,0.01,0.90,0.01,0.90,0.01,0.93,0.01,1.31,0.01,64.02,0.61,61.90,0.52,62.63,0.5,18.42,0.2,16.48,0.19,15.64,0.17,0.50,0.01,0.61,0.0,0.96,0.01,9.15,0.11,9.99,0.08,16.52,0.17
4,DLHCX,Delaware Healthcare Fund Class C,Delaware Funds by Macquarie,2010-01-27,Health,3,3,3,The investment seeks maximum long-term capital...,Blend,Large,USD,2.02,1.21,0.00,100.00,0.0,0.00,0.00,0.0,17.88,3.07,2.17,10.85,31584.01,0.1

### ***Analysis Topics:***

### **1) Fund Family:**




1. top ten fund families that have issued the most funds in the past ten years

We filtered the data to find the top ten fund families with most funds issued in past ten years. The result shows that Fidelity Investments has issued 502 mutual funds, surpass the second place which is American Funds with 66 mutual funds. Obviously, the top ten funds show below have larger market share and have more investment choice among the many fund families.

In [ ]:
%%bigquery --project=ba775-project-team3
select fund_family,count(fund_family) as family_pop
from `ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
where extract(year from inception_date) between 2011 and 2020
group by fund_family
order by family_pop desc limit 10

,fund_family,family_pop
0,Fidelity Investments,502
1,American Funds,436
2,Franklin Templeton Investments,373
3,Invesco,362
4,Columbia,337
5,John Hancock,326
6,JPMorgan,292
7,Goldman Sachs,286
8,MFS,264
9,PGIM Funds (Prudential),244


2. Size type of Mutual Funds vs. Rating

We analyzed the relationships between size type and rating in the following table. We found that the mutual funds with small size are generally have lower rating, return rating, and risk rating than large and medium size. The funds with midium size have highest return rating in average, and the funds with large size have lowest risk rating in average.

In [ ]:
%%bigquery --project=ba775-project-team3
select size_type,count(size_type)as size_pop, avg(rating) average_rating, avg(return_rating) average_return_rating, avg(risk_rating) avg_risk_rating
from `ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
group by size_type
order by size_pop desc
limit 3



,size_type,size_pop,average_rating,average_return_rating,avg_risk_rating
0,Large,13857,3.046949,3.026078,3.006320
1,Medium,6144,3.055704,3.057410,3.163109
2,Small,3625,3.004258,3.017619,3.215725


### **2) Investment Types & Strategies:**

3. Which investment type (growth, value, blend, others) has produced the high returns in the last three years?

In [ ]:
%%bigquery --project=ba775-project-team3
SELECT investment_type, count(investment_type) number_of_funds
FROM `ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
GROUP BY investment_type
ORDER BY number_of_funds DESC
LIMIT 3

,investment_type,number_of_funds
0,Blend,10990
1,Value,7321
2,Growth,5315


In [ ]:
%%bigquery --project=ba775-project-team3
SELECT 
  investment_type, count(investment_type) number_of_funds, 
  avg(fund_return_3months) as average_return_3months, 
  avg(fund_return_3years) as average_return_3years,
  avg(fund_return_5years) as average_return_5years,
  avg(fund_return_10years) as average_return_10years,
FROM `ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
GROUP BY investment_type
ORDER BY number_of_funds DESC
LIMIT 3

,investment_type,number_of_funds,average_return_3months,average_return_3years,average_return_5years,average_return_10years
0,Blend,10990,0.392806,3.866312,5.579253,6.459579
1,Value,7321,-0.217132,0.363691,2.990715,4.936459
2,Growth,5315,2.345962,10.359763,10.722378,10.238240


4. Does the investment type with highest return has lower sharpe ratio? The impact of size_type on the return in 5 years?

In [ ]:
%%bigquery --project=ba775-project-team3
SELECT 
  investment_type, count(investment_type) number_of_funds, 
  avg(fund_return_3months) as average_return_3months, 
  avg(fund_return_3years) as average_return_3years, avg(fund_sharpe_ratio_3years) as average_sharpe_ratio_3years, 
  avg(fund_return_5years) as average_return_5years, avg(fund_sharpe_ratio_5years) as average_sharpe_ratio_5years, 
  avg(fund_return_10years) as average_return_10years, avg(fund_sharpe_ratio_10years) as average_sharp_ratio_10years
FROM `ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
GROUP BY investment_type
ORDER BY number_of_funds DESC
LIMIT 3

,investment_type,number_of_funds,average_return_3months,average_return_3years,average_sharpe_ratio_3years,average_return_5years,average_sharpe_ratio_5years,average_return_10years,average_sharp_ratio_10years
0,Blend,10990,0.392806,3.866312,0.301231,5.579253,0.478038,6.459579,0.640288
1,Value,7321,-0.217132,0.363691,0.114385,2.990715,0.327708,4.936459,0.556244
2,Growth,5315,2.345962,10.359763,0.537733,10.722378,0.654328,10.238240,0.715199


5. Which investment strategy has the highest price_earnings_ratio?

In [ ]:
%%bigquery --project=ba775-project-team3
select investment_strategy,investment_type,size_type
from `ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
order by price_earnings_ratio desc limit 10

,investment_strategy,investment_type,size_type
0,The investment seeks total return by investing...,Growth,Small
1,The investment seeks total return by investing...,Growth,Small
2,The investment seeks total return by investing...,Growth,Small
3,The investment seeks to provide total return. ...,Value,Large
4,The investment seeks total return by investing...,Growth,Small
5,The investment seeks to generate current incom...,Value,Medium
6,The investment seeks to generate current incom...,Value,Medium
7,The investment seeks to generate current incom...,Value,Medium
8,The investment seeks to generate current incom...,Value,Medium
9,The investment seeks total return by investing...,Growth,Small


6. What are the top 10 holdings of the top ten funds by market cap?

In [ ]:
%%bigquery --project=ba775-project-team3
select fund_extended_name,fund_family,median_market_cap,top10_holdings
from`ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
order by median_market_cap desc limit 10

,fund_extended_name,fund_family,median_market_cap,top10_holdings
0,Voya Russell Large Cap Growth Index Portfolio ...,Voya,408144.41,"Apple Inc: 13.56, Microsoft Corp: 11.43, Amazo..."
1,Voya Russell Large Cap Growth Index Portfolio ...,Voya,408144.41,"Apple Inc: 13.56, Microsoft Corp: 11.43, Amazo..."
2,Voya Russell Large Cap Growth Index Portfolio ...,Voya,408144.41,"Apple Inc: 13.56, Microsoft Corp: 11.43, Amazo..."
3,Rational Iron Horse Fund Class A,Rational Funds,405325.03,"Amazon.com Inc: 7.70, 3M Co: 5.20, JPMorgan Ch..."
4,Rational Iron Horse Fund Class C,Rational Funds,405325.03,"Amazon.com Inc: 7.70, 3M Co: 5.20, JPMorgan Ch..."
5,Rational Iron Horse Fund Class I,Rational Funds,405325.03,"Amazon.com Inc: 7.70, 3M Co: 5.20, JPMorgan Ch..."
6,BlackRock Exchange Portfolio BlackRock Shares,BlackRock,390240.44,"Microsoft Corp: 32.28, Berkshire Hathaway Inc ..."
7,Shelton Capital Management Nasdaq-100 Index Fu...,Shelton Capital Management,378044.81,"Apple Inc: 14.12, Amazon.com Inc: 10.74, Micro..."
8,Shelton Capital Management Nasdaq-100 Index Fu...,Shelton Capital Management,378044.81,"Apple Inc: 14.12, Amazon.com Inc: 10.74, Micro..."
9,Vanguard Mega Cap Growth Index Fund Institutio...,Vanguard,375297.41,"Apple Inc: 12.93, Microsoft Corp: 11.54, Amazo..."


7. List all the funds with highest, medium, lowest risk ratings as "Highest Risk","Medium Risk", "Lowest Risk" respectively.

In [ ]:
%%bigquery --project=ba775-project-team3
select fund_family, fund_symbol, risk_rating,
CASE WHEN risk_rating in (4,5) THEN 'Highest Risk'
WHEN risk_rating in (2,3) THEN 'Medium Risk'
ELSE 'Lowest Risk' END AS category
from`ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
order by risk_rating desc limit 10

,fund_family,fund_symbol,risk_rating,category
0,Fidelity Investments,FBIOX,5,Highest Risk
1,Eventide Funds,ETAHX,5,Highest Risk
2,Fidelity Investments,FBTCX,5,Highest Risk
3,Franklin Templeton Investments,FBDIX,5,Highest Risk
4,Eventide Funds,ETCHX,5,Highest Risk
5,Fidelity Investments,FBTTX,5,Highest Risk
6,Fidelity Investments,FBTIX,5,Highest Risk
7,Fidelity Investments,FBTAX,5,Highest Risk
8,Eventide Funds,ETNHX,5,Highest Risk
9,Eventide Funds,ETIHX,5,Highest Risk


8. List the top 10 funds with the highest price_cashflow ratios.

In [ ]:
%%bigquery --project=ba775-project-team3
select fund_extended_name, fund_symbol, price_cashflow_ratio
from`ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
order by price_cashflow_ratio desc limit 10

,fund_extended_name,fund_symbol,price_cashflow_ratio
0,Fairholme Fund,FAIRX,40.00
1,Westwood Alternative Income Fund Ultra Shares,WMNUX,38.61
2,Westwood Alternative Income Fund Institutional...,WMNIX,38.61
3,Zevenbergen Genea Fund Investor Class,ZVGNX,38.60
4,Zevenbergen Genea Fund Institutional Class,ZVGIX,38.60
5,Transamerica Capital Growth Fund Class C,ILLLX,38.45
6,Transamerica Capital Growth Fund Class A,IALAX,38.45
7,Morgan Stanley Institutional Fund Trust Discov...,MACGX,38.10
8,Morgan Stanley Institutional Fund Trust Discov...,MSKLX,38.10
9,Morgan Stanley Institutional Fund Trust Discov...,MPEGX,38.10


### **3) Fund Returns:**

9. Which fund has the maximum fund_return in the past 3,5,10 years respectively.

In [ ]:
%%bigquery --project=ba775-project-team3
select fund_extended_name, fund_symbol, fund_return_3years, fund_return_5years, fund_return_10years
from`ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
WHERE fund_return_10years IS NOT NULL
order by fund_return_3years desc, fund_return_5years desc, fund_return_10years desc limit 20

,fund_extended_name,fund_symbol,fund_return_3years,fund_return_5years,fund_return_10years
0,Morgan Stanley Institutional Fund Trust Discov...,MMCGX,47.39,30.78,18.33
1,Morgan Stanley Institutional Fund Trust Discov...,MPEGX,47.27,30.69,18.25
2,Morgan Stanley Institutional Fund Trust Discov...,MACGX,46.86,30.32,17.93
3,Morgan Stanley Institutional Fund Trust Discov...,MSKLX,46.14,29.59,17.30
4,Morgan Stanley Institutional Fund Trust Discov...,MSMFX,45.67,29.30,17.03
5,Morgan Stanley Insight Fund Class I,CPODX,40.95,31.43,22.11
6,Morgan Stanley Insight Fund Class IS,MCRTX,40.94,31.47,22.14
7,Morgan Stanley Insight Fund Class A,CPOAX,40.56,31.05,21.75
8,Morgan Stanley Insight Fund Class L,CPOCX,39.89,30.40,21.07
9,Virtus Zevenbergen Innovative Growth Stock Fun...,SCATX,39.86,28.72,20.52



10. Top ten funds with highest return in 3 months



In [ ]:
%%bigquery --project=ba775-project-team3
select fund_extended_name, fund_symbol, fund_return_3months,sector_basic_materials, sector_financial_services, sector_real_estate, sector_healthcare, sector_utilities, sector_communication_services, sector_energy, sector_industrials, sector_technology
from`ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
WHERE fund_return_3months IS NOT NULL
order by fund_return_3months desc
limit 10

,fund_extended_name,fund_symbol,fund_return_3months,sector_basic_materials,sector_financial_services,sector_real_estate,sector_healthcare,sector_utilities,sector_communication_services,sector_energy,sector_industrials,sector_technology
0,Hodges Fund Institutional Class,HDPIX,54.40,18.77,1.71,0.00,6.35,0.00,7.65,13.54,7.02,25.13
1,Schneider Small Cap Value Fund,SCMVX,43.16,17.37,21.26,6.30,0.72,0.00,0.05,13.43,21.12,12.46
2,ICON Opportunities Fund,ICONX,41.46,7.29,12.07,0.03,0.08,0.01,0.01,0.01,27.29,23.79
3,Hartford Growth Opportunities HLS Fund Class IA,HAGOX,39.49,0.03,4.51,2.02,17.74,0.00,14.45,0.09,2.37,37.24
4,Hartford Growth Opportunities HLS Fund Class IB,HBGOX,39.38,0.03,4.51,2.02,17.74,0.00,14.45,0.09,2.37,37.24
5,Hartford Growth Opportunities HLS Fund Class IC,HCGOX,39.29,0.03,4.51,2.02,17.74,0.00,14.45,0.09,2.37,37.24
6,ICON Consumer Discretionary Fund Class S,ICCCX,35.82,0.00,4.32,0.00,0.00,0.00,8.54,0.00,2.90,0.00
7,ICON Consumer Discretionary Fund Class A,ICCAX,35.68,0.00,4.32,0.00,0.00,0.00,8.54,0.00,2.90,0.00
8,ICON Energy Fund Class S,ICENX,35.65,7.92,0.00,0.00,0.00,0.00,0.00,83.44,8.64,0.00
9,ICON Energy Fund Class A,ICEAX,35.60,7.92,0.00,0.00,0.00,0.00,0.00,83.44,8.64,0.00


11. Top ten funds with highest return in 3 years

In [ ]:
%%bigquery --project=ba775-project-team3
select fund_extended_name, fund_symbol, fund_return_3years,sector_basic_materials, sector_financial_services, sector_real_estate, sector_healthcare, sector_utilities, sector_communication_services, sector_energy, sector_industrials, sector_technology
from`ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
WHERE fund_return_3years IS NOT NULL
order by fund_return_3years desc
limit 10

,fund_extended_name,fund_symbol,fund_return_3years,sector_basic_materials,sector_financial_services,sector_real_estate,sector_healthcare,sector_utilities,sector_communication_services,sector_energy,sector_industrials,sector_technology
0,Morgan Stanley Institutional Fund Trust Discov...,MMCGX,47.39,0.19,0.00,0.00,15.59,0.0,23.90,0.24,0.0,45.35
1,Morgan Stanley Institutional Fund Trust Discov...,MPEGX,47.27,0.19,0.00,0.00,15.59,0.0,23.90,0.24,0.0,45.35
2,Morgan Stanley Institutional Fund Trust Discov...,MACGX,46.86,0.19,0.00,0.00,15.59,0.0,23.90,0.24,0.0,45.35
3,Morgan Stanley Institutional Fund Trust Discov...,MSKLX,46.14,0.19,0.00,0.00,15.59,0.0,23.90,0.24,0.0,45.35
4,Morgan Stanley Institutional Fund Trust Discov...,MSMFX,45.67,0.19,0.00,0.00,15.59,0.0,23.90,0.24,0.0,45.35
5,Morgan Stanley Insight Fund Class I,CPODX,40.95,0.20,0.00,0.00,20.28,0.0,22.11,0.25,0.0,42.49
6,Morgan Stanley Insight Fund Class IS,MCRTX,40.94,0.20,0.00,0.00,20.28,0.0,22.11,0.25,0.0,42.49
7,Morgan Stanley Insight Fund Class A,CPOAX,40.56,0.20,0.00,0.00,20.28,0.0,22.11,0.25,0.0,42.49
8,Morgan Stanley Insight Fund Class L,CPOCX,39.89,0.20,0.00,0.00,20.28,0.0,22.11,0.25,0.0,42.49
9,Virtus Zevenbergen Innovative Growth Stock Fun...,SCATX,39.86,0.00,3.78,1.92,16.32,0.0,14.40,0.00,0.0,37.34


12. Top ten funds with highest return in 10 years

In [ ]:
%%bigquery --project=ba775-project-team3
select fund_extended_name, fund_symbol, fund_return_10years,sector_basic_materials, sector_financial_services, sector_real_estate, sector_healthcare, sector_utilities, sector_communication_services, sector_energy, sector_industrials, sector_technology
from`ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
WHERE fund_return_10years IS NOT NULL and sector_basic_materials is not null
order by fund_return_10years desc
limit 10

,fund_extended_name,fund_symbol,fund_return_10years,sector_basic_materials,sector_financial_services,sector_real_estate,sector_healthcare,sector_utilities,sector_communication_services,sector_energy,sector_industrials,sector_technology
0,Rydex NASDAQ-100 2x Strategy Fund Class A,RYVLX,32.79,0.0,1.99,0.27,7.61,0.66,20.14,0.0,2.63,45.07
1,Rydex NASDAQ-100 2x Strategy Fund Class H,RYVYX,32.78,0.0,1.99,0.27,7.61,0.66,20.14,0.0,2.63,45.07
2,ProFunds UltraNASDAQ-100 Fund Investor Class,UOPIX,32.61,0.0,2.08,0.00,7.46,0.67,20.10,0.0,2.52,44.50
3,Rydex NASDAQ-100 2x Strategy Fund Class C,RYCCX,31.78,0.0,1.99,0.27,7.61,0.66,20.14,0.0,2.63,45.07
4,ProFunds UltraNASDAQ-100 Fund Service Class,UOPSX,31.30,0.0,2.08,0.00,7.46,0.67,20.10,0.0,2.52,44.50
5,ProFunds Internet UltraSector Fund Investor Class,INPIX,25.62,0.0,8.49,0.00,5.32,0.00,30.06,0.0,0.00,37.89
6,ProFunds Internet UltraSector Fund Service Class,INPSX,24.37,0.0,8.49,0.00,5.32,0.00,30.06,0.0,0.00,37.89
7,ProFunds Semiconductor UltraSector Fund Invest...,SMPIX,24.17,0.0,0.00,0.00,0.00,0.00,0.13,0.0,0.00,99.87
8,ProFunds Technology UltraSector Fund Investor ...,TEPIX,23.73,0.0,0.00,0.00,0.67,0.00,19.68,0.0,0.17,78.85
9,ProFunds Semiconductor UltraSector Fund Servic...,SMPSX,22.93,0.0,0.00,0.00,0.00,0.00,0.13,0.0,0.00,99.87


### **4) Financial Ratios:**

13. Which is the most popular fund category?


In [ ]:
%%bigquery --project=ba775-project-team3
SELECT category, count(category) number_of_funds
FROM `ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
GROUP BY category
ORDER BY number_of_funds DESC 
LIMIT 10

,category,number_of_funds
0,Large Growth,1302
1,Large Blend,1251
2,Large Value,1132
3,World Large Stock,836
4,Diversified Emerging Mkts,773
5,Foreign Large Blend,691
6,High Yield Bond,660
7,Small Blend,645
8,Allocation - 50% to 70% Equity,638
9,Small Growth,619


14. Top ten funds with highest alpha in 5 years

In [ ]:
%%bigquery --project=ba775-project-team3
select fund_extended_name, fund_symbol, fund_alpha_5years,sector_basic_materials, sector_financial_services, sector_real_estate, sector_healthcare, sector_utilities, sector_communication_services, sector_energy, sector_industrials, sector_technology
from`ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
WHERE fund_alpha_5years IS NOT NULL
order by fund_alpha_5years desc
limit 10

,fund_extended_name,fund_symbol,fund_alpha_5years,sector_basic_materials,sector_financial_services,sector_real_estate,sector_healthcare,sector_utilities,sector_communication_services,sector_energy,sector_industrials,sector_technology
0,ProFunds Precious Metals UltraSector Fund Inve...,PMPIX,25.11,100.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
1,ProFunds Precious Metals UltraSector Fund Serv...,PMPSX,24.13,100.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
2,Baillie Gifford Long Term Global Growth Fund C...,BGLTX,21.29,0.0,0.9,0.59,10.4,0.0,22.0,0.0,0.0,18.24
3,Baillie Gifford Long Term Global Growth Fund C...,BGLKX,21.24,0.0,0.9,0.59,10.4,0.0,22.0,0.0,0.0,18.24
4,Baillie Gifford Long Term Global Growth Fund C...,BSGLX,21.21,0.0,0.9,0.59,10.4,0.0,22.0,0.0,0.0,18.24
5,OCM Gold Fund Advisors Class,OCMAX,20.00,100.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
6,Baron Global Advantage Fund R6 Shares,BGLUX,19.88,0.0,0.0,0.00,20.6,0.0,13.0,0.0,0.0,41.26
7,Baron Global Advantage Fund Institutional Shares,BGAIX,19.87,0.0,0.0,0.00,20.6,0.0,13.0,0.0,0.0,41.26
8,Baron Global Advantage Fund Retail Shares,BGAFX,19.63,0.0,0.0,0.00,20.6,0.0,13.0,0.0,0.0,41.26
9,OCM Gold Fund Investor Class,OCMGX,19.17,100.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00


15. Top ten funds with highest alpha in 10 years

In [ ]:
%%bigquery --project=ba775-project-team3
select fund_extended_name, fund_symbol, fund_alpha_10years,sector_basic_materials, sector_financial_services, sector_real_estate, sector_healthcare, sector_utilities, sector_communication_services, sector_energy, sector_industrials, sector_technology
from`ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
WHERE fund_alpha_10years IS NOT NULL
order by fund_alpha_10years desc
limit 10

,fund_extended_name,fund_symbol,fund_alpha_10years,sector_basic_materials,sector_financial_services,sector_real_estate,sector_healthcare,sector_utilities,sector_communication_services,sector_energy,sector_industrials,sector_technology
0,ProFunds UltraShort Latin America Fund Investo...,UFPIX,28.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ProFunds UltraShort Latin America Fund Service...,UFPSX,27.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Morgan Stanley Institutional Fund, Inc. Global...",MGTSX,13.74,0.53,14.69,0.00,2.4,0.0,19.29,0.0,4.13,28.28
3,"Morgan Stanley Institutional Fund, Inc. Global...",MGORX,13.74,0.53,14.69,0.00,2.4,0.0,19.29,0.0,4.13,28.28
4,"Morgan Stanley Institutional Fund, Inc. Global...",MGGIX,13.72,0.53,14.69,0.00,2.4,0.0,19.29,0.0,4.13,28.28
5,"Morgan Stanley Institutional Fund, Inc. Global...",MGGPX,13.41,0.53,14.69,0.00,2.4,0.0,19.29,0.0,4.13,28.28
6,"Morgan Stanley Institutional Fund, Inc. Global...",MGGLX,13.33,0.53,14.69,0.00,2.4,0.0,19.29,0.0,4.13,28.28
7,"Morgan Stanley Institutional Fund, Inc. Global...",MSOPX,12.69,0.53,14.69,0.00,2.4,0.0,19.29,0.0,4.13,28.28
8,T. Rowe Price Global Technology Fund I Class,PGTIX,12.09,0.00,4.09,1.49,0.0,0.0,18.52,0.0,0.00,62.18
9,T. Rowe Price Global Technology Fund,PRGTX,12.04,0.00,4.09,1.49,0.0,0.0,18.52,0.0,0.00,62.18


16. Which kind of investment size type has the most mean annual return in 5 years?(Does size matter in the long run?)

In [ ]:
%%bigquery --project=ba775-project-team3
SELECT size_type, count(size_type) as number_of_fund, avg(fund_mean_annual_return_3years) as average_3years_return
FROM `ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
group by size_type 
having size_type is not null
order by average_3years_return DESC

,size_type,number_of_fund,average_3years_return
0,Large,13857,0.514192
1,Medium,6144,0.371306
2,Small,3625,0.275163


In [ ]:
%%bigquery --project=ba775-project-team3
SELECT size_type, count(size_type) as number_of_fund, avg(fund_mean_annual_return_5years) as average_5years_return
FROM `ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
group by size_type 
having size_type is not null
order by average_5years_return DESC

,size_type,number_of_fund,average_5years_return
0,Large,13857,0.625256
1,Small,3625,0.478457
2,Medium,6144,0.436058


In [ ]:
%%bigquery --project=ba775-project-team3
SELECT size_type, count(size_type) as number_of_fund, avg(fund_mean_annual_return_10years) as average_10years_return
FROM `ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
group by size_type 
having size_type is not null
order by average_10years_return DESC

,size_type,number_of_fund,average_10years_return
0,Large,13857,0.684177
1,Small,3625,0.601599
2,Medium,6144,0.509060


17. Which fund_family made investment with stable profit? (according to the std)

In [ ]:
%%bigquery --project=ba775-project-team3
SELECT fund_family, count(fund_family) as number_of_fund, stddev(fund_mean_annual_return_3years) as STD_3years
FROM `ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
group by fund_family
having fund_family is not null
and STD_3years is not null
order by STD_3years 
limit 10

,fund_family,number_of_fund,STD_3years
0,Insignia Macro,2,0.000000
1,Harvest Funds,2,0.007071
2,Prospector Funds,2,0.007071
3,RWC,2,0.007071
4,Becker,2,0.007071
5,Kellner,2,0.007071
6,AAAMCO,2,0.007071
7,Fulcrum,2,0.007071
8,Superfund,2,0.007071
9,Selective,2,0.007071


In [ ]:
%%bigquery --project=ba775-project-team3
SELECT fund_family, count(fund_family) as number_of_fund, stddev(fund_mean_annual_return_5years) as STD_5years
FROM `ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
group by fund_family
having fund_family is not null
and STD_5years is not null
order by STD_5years 
limit 10

,fund_family,number_of_fund,STD_5years
0,Fulcrum,2,0.000000
1,Neiman Funds,3,0.000000
2,Acadian Funds,3,0.005774
3,Coho,2,0.007071
4,Insignia Macro,2,0.007071
5,Seafarer Funds,4,0.007071
6,ATAC Fund,2,0.007071
7,Clarkston Funds,4,0.007071
8,Logan Capital,2,0.007071
9,Leavell,2,0.007071


In [ ]:
%%bigquery --project=ba775-project-team3
SELECT fund_family, count(fund_family) as number_of_fund, stddev(fund_mean_annual_return_10years) as STD_10years
FROM `ba775-project-team3.Mutual_Funds.Mutual_funds_rawdata`
group by fund_family
having fund_family is not null
and STD_10years is not null
order by STD_10years 
limit 10

,fund_family,number_of_fund,STD_10years
0,Neiman Funds,3,0.000000
1,Motley Fool,4,0.000000
2,Walthausen Funds,5,0.000000
3,Leavell,2,0.000000
4,Acadian Funds,3,0.005774
5,Sound Shore,2,0.007071
6,Euro Pacific Asset Management,10,0.007071
7,AAM,15,0.007071
8,Westchester Capital,6,0.007071
9,Becker,2,0.007071
